<a href="https://colab.research.google.com/github/NerusuSahithi011/NLP/blob/main/lab_assignment07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
data=pd.read_csv('/content/Lab7_Text_Similarity_Dataset.csv')
print(data)

    ID    Category                                               Text
0    1      Sports  The cricket team won the championship after a ...
1    2      Sports  Football players trained hard for the internat...
2    3      Sports  The athlete broke the Olympic record in the 10...
3    4      Sports  The tennis match was exciting and full of inte...
4    5      Sports  Basketball teams compete for the national trop...
5    6      Sports  The coach motivated the players before the big...
6    7    Politics  The government passed a new economic reform bill.
7    8    Politics  The president addressed the nation about forei...
8    9    Politics  Parliament debated the healthcare policy for s...
9   10    Politics  The election results were announced yesterday ...
10  11    Politics  Political leaders held a meeting to discuss re...
11  12    Politics  The prime minister introduced new development ...
12  13      Health  Doctors recommend regular exercise for good he...
13  14      Health  

IMPORT LIBRARIES

In [4]:
# Text preprocessing
import re
import string

# Data handling
import numpy as np
import pandas as pd

# TF-IDF & Cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# NLTK for preprocessing & WordNet
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn


In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Preprocessing Function

In [9]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = text.lower()

    # Remove punctuation & numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return " ".join(tokens)

# Apply preprocessing
data["Processed_Text"] = data["Text"].apply(preprocess)

print(data.head())

   ID Category                                               Text  \
0   1   Sports  The cricket team won the championship after a ...   
1   2   Sports  Football players trained hard for the internat...   
2   3   Sports  The athlete broke the Olympic record in the 10...   
3   4   Sports  The tennis match was exciting and full of inte...   
4   5   Sports  Basketball teams compete for the national trop...   

                                      Processed_Text  
0    cricket team championship thrilling final match  
1  football player trained hard international tou...  
2            athlete broke olympic record meter race  
3           tennis match exciting full intense rally  
4  basketball team compete national trophy every ...  


Bag-of-Words Representation

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(data["Processed_Text"])

print("BoW Matrix Shape:", bow_matrix.shape)

BoW Matrix Shape: (24, 126)


Cosine Similarity using BoW

In [12]:
cos_sim = cosine_similarity(bow_matrix)

cosine_df = pd.DataFrame(cos_sim)
print("Cosine Similarity Matrix (first 5 rows):")
print(cosine_df.head())


Cosine Similarity Matrix (first 5 rows):
         0    1    2         3         4         5    6    7    8    9   ...  \
0  1.000000  0.0  0.0  0.166667  0.154303  0.000000  0.0  0.0  0.0  0.0  ...   
1  0.000000  1.0  0.0  0.000000  0.000000  0.182574  0.0  0.0  0.0  0.0  ...   
2  0.000000  0.0  1.0  0.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  ...   
3  0.166667  0.0  0.0  1.000000  0.000000  0.000000  0.0  0.0  0.0  0.0  ...   
4  0.154303  0.0  0.0  0.000000  1.000000  0.000000  0.0  0.0  0.0  0.0  ...   

    14   15   16   17   18   19   20   21   22   23  
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 24 columns]


Example comparison

In [14]:
i = 0
j = 1

print("Text 1:", data["Text"][i])
print("Text 2:", data["Text"][j])
print("Cosine Similarity (BoW):", cos_sim[i][j])

Text 1: The cricket team won the championship after a thrilling final match.
Text 2: Football players trained hard for the international tournament.
Cosine Similarity (BoW): 0.0


Jaccard Similarity

In [16]:
def jaccard_similarity(text1, text2):
    set1 = set(text1.split())
    set2 = set(text2.split())
    return len(set1 & set2) / len(set1 | set2)

for i in range(5):
    score = jaccard_similarity(data["Processed_Text"][i], data["Processed_Text"][i+1])
    print(f"Jaccard Similarity between Doc {i} and {i+1}: {score}")

Jaccard Similarity between Doc 0 and 1: 0.0
Jaccard Similarity between Doc 1 and 2: 0.0
Jaccard Similarity between Doc 2 and 3: 0.0
Jaccard Similarity between Doc 3 and 4: 0.0
Jaccard Similarity between Doc 4 and 5: 0.0


WordNet Semantic Similarity

In [17]:
def word_similarity(w1, w2):
    syn1 = wn.synsets(w1)
    syn2 = wn.synsets(w2)
    if syn1 and syn2:
        return syn1[0].wup_similarity(syn2[0])
    return 0

def sentence_similarity(sent1, sent2):
    words1 = sent1.split()
    words2 = sent2.split()
    scores = []

    for w1 in words1:
        for w2 in words2:
            sim = word_similarity(w1, w2)
            if sim is not None:
                scores.append(sim)

    if len(scores) > 0:
        return sum(scores) / len(scores)
    return 0


In [19]:
i = 12
j = 17

print("Text 1:", data["Text"][i])
print("Text 2:", data["Text"][j])

print("Cosine (BoW):", cos_sim[i][j])
print("Jaccard:", jaccard_similarity(data["Processed_Text"][i], data["Processed_Text"][j]))
print("WordNet:", sentence_similarity(data["Processed_Text"][i], data["Processed_Text"][j]))

Text 1: Doctors recommend regular exercise for good health.
Text 2: Healthy habits reduce the risk of chronic illness.
Cosine (BoW): 0.0
Jaccard: 0.0
WordNet: 0.19745888833866132
